# 20210905

## Install dependency
~~~
Based on the pyproject.toml in the current path, dependency will be installed in order.
~~~

In [ ]:
cd /content/drive/MyDrive/ColabNotebooks/Research/KG2QGwithT5/adjacencyattentionwithoutselfloop

/content/drive/MyDrive/ColabNotebooks/Research/KG2QGwithT5/adjacencyattentionwithoutselfloop


In [ ]:
!poetry lock

/bin/bash: poetry: command not found


In [ ]:
!pip install -q --pre poetry==1.1.8

     |████████████████████████████████| 173 kB 9.4 MB/s 
     |████████████████████████████████| 91 kB 13.8 MB/s 
     |████████████████████████████████| 40 kB 7.2 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 420 kB 39.9 MB/s 
     |████████████████████████████████| 5.3 MB 52.5 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 3.0 MB 68.5 MB/s 
     |████████████████████████████████| 338 kB 78.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install --requirement <(poetry export --format requirements.txt)

Ignoring colorama: markers 'python_version >= "3.6" and python_full_version < "3.0.0" and platform_system == "Windows" or python_full_version >= "3.5.0" and python_version >= "3.6" and platform_system == "Windows"' don't match your environment
Ignoring pywin32: markers 'sys_platform == "win32" and python_version >= "3.6"' don't match your environment
Ignoring waitress: markers 'platform_system == "Windows" and python_version >= "3.6" and python_full_version >= "3.6.0"' don't match your environment
     |████████████████████████████████| 132 kB 7.2 MB/s 
     |████████████████████████████████| 1.3 MB 55.8 MB/s 
     |████████████████████████████████| 1.1 MB 62.8 MB/s 
     |████████████████████████████████| 112 kB 65.2 MB/s 
     |████████████████████████████████| 178 kB 61.5 MB/s 
     |████████████████████████████████| 97 kB 8.9 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 145 kB 62.5 MB/s 
     |████████████████████████████████

## Import Library

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
 
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

import io
import string
from collections import Counter, defaultdict, OrderedDict
from nltk.tokenize import wordpunct_tokenize, word_tokenize
 
import gc

from AdjacencyAttentionWithoutSelfloopTransformers import (
  AdamW,
  T5ForConditionalGeneration,
  T5Tokenizer,
  get_linear_schedule_with_warmup
)

import os.path

import wandb
wandb.login()

wandb: Currently logged in as: macho000 (use `wandb login --relogin` to force relogin)


True

## Preprocess

In [ ]:
def preprocess_wq(config):
    if not os.path.isfile("processed/mhqg-wq/SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-wq/SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-wq/SOURCE_MASK_memmap.npy"
        SOURCE_CROSS_MASK_PATH = "processed/mhqg-wq/SOURCE_CROSS_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-wq/TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-wq/TARGET_MASK_memmap.npy"
        # input_length = 18624
        input_length = 18989
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        SOURCE_CROSS_MASK_memmap = np.memmap(
        filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del SOURCE_CROSS_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        with open("data/mhqg-wq/train.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += (x + " ")
                target = jo['outSeq']
                n = len(jo['inGraph']['g_node_names'])
                adj_matrix = {}
                adj_matrix2 = {}
                graph = {'node_name_id2word':{}}
                for idx, nid in enumerate(jo['inGraph']['g_node_names']):
                    # replace answer token to "<answer> answer <answer>"
                    if jo['inGraph']['g_node_names'][nid] in answers:
                        graph['node_name_id2word'][nid] = "<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"
                    else:
                        graph['node_name_id2word'][nid] = jo['inGraph']['g_node_names'][nid]

                    # create adj_matrix with 0 value
                    for idy, nid2 in enumerate(jo['inGraph']['g_node_names']):
                        if jo['inGraph']['g_node_names'][nid2] in answers:
                            adj_matrix2["<answer> "+ jo['inGraph']['g_node_names'][nid2] + " <answer>"] = 0
                        else:
                            adj_matrix2[jo['inGraph']['g_node_names'][nid2]] = 0
                    if jo['inGraph']['g_node_names'][nid] in answers:
                        adj_matrix["<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"] = adj_matrix2.copy()
                    else:
                        adj_matrix[jo['inGraph']['g_node_names'][nid]] = adj_matrix2.copy()
                for nid, val in jo['inGraph']['g_adj'].items():
                    for nid2, edge in val.items():
                        edge = edge.split('/')[-1]
                        adj_matrix[graph["node_name_id2word"][nid]][graph["node_name_id2word"][nid2]] = 1
                input_knowledge_graph = ""
                for i,ids in enumerate(list(adj_matrix.keys())):
                    input_knowledge_graph += (ids + " ")

                input, target = input_knowledge_graph, target

                tokenized_row_input = ""
                tokenized_row_input = tokenizer.tokenize(input)
                if len(tokenized_row_input) > 512:
                    continue

                tokenized_inputs = tokenizer.batch_encode_plus_self_attention(
                    [input_knowledge_graph], adjacancy_matrix=adj_matrix, max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )

                SOURCE_CROSS_ATTENTION_MASK_memmap = np.memmap(
                    filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512)
                )

                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()
                SOURCE_CROSS_ATTENTION_MASK_memmap[list_index] = tokenized_inputs["cross_attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap
                del SOURCE_CROSS_ATTENTION_MASK_memmap
                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

    if not os.path.isfile("processed/mhqg-wq/VAL_SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-wq/VAL_SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-wq/VAL_SOURCE_MASK_memmap.npy"
        SOURCE_CROSS_MASK_PATH = "processed/mhqg-wq/VAL_SOURCE_CROSS_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-wq/VAL_TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-wq/VAL_TARGET_MASK_memmap.npy"
        # input_length = 18624
        input_length = 2000
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        SOURCE_CROSS_MASK_memmap = np.memmap(
        filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del SOURCE_CROSS_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        with open("data/mhqg-wq/dev.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += (x + " ")
                target = jo['outSeq']
                n = len(jo['inGraph']['g_node_names'])
                adj_matrix = {}
                adj_matrix2 = {}
                graph = {'node_name_id2word':{}}
                for idx, nid in enumerate(jo['inGraph']['g_node_names']):
                    # replace answer token to "<answer> answer <answer>"
                    if jo['inGraph']['g_node_names'][nid] in answers:
                        graph['node_name_id2word'][nid] = "<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"
                    else:
                        graph['node_name_id2word'][nid] = jo['inGraph']['g_node_names'][nid]

                    # create adj_matrix with 0 value
                    for idy, nid2 in enumerate(jo['inGraph']['g_node_names']):
                        if jo['inGraph']['g_node_names'][nid2] in answers:
                            adj_matrix2["<answer> "+ jo['inGraph']['g_node_names'][nid2] + " <answer>"] = 0
                        else:
                            adj_matrix2[jo['inGraph']['g_node_names'][nid2]] = 0
                    if jo['inGraph']['g_node_names'][nid] in answers:
                        adj_matrix["<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"] = adj_matrix2.copy()
                    else:
                        adj_matrix[jo['inGraph']['g_node_names'][nid]] = adj_matrix2.copy()
                for nid, val in jo['inGraph']['g_adj'].items():
                    for nid2, edge in val.items():
                        edge = edge.split('/')[-1]
                        adj_matrix[graph["node_name_id2word"][nid]][graph["node_name_id2word"][nid2]] = 1
                input_knowledge_graph = ""
                for i,ids in enumerate(list(adj_matrix.keys())):
                    input_knowledge_graph += (ids + " ")

                input, target = input_knowledge_graph, target

                tokenized_row_input = ""
                tokenized_row_input = tokenizer.tokenize(input)
                if len(tokenized_row_input) > 512:
                    continue

                tokenized_inputs = tokenizer.batch_encode_plus_self_attention(
                    [input_knowledge_graph], adjacancy_matrix=adj_matrix, max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )

                SOURCE_CROSS_ATTENTION_MASK_memmap = np.memmap(
                    filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512)
                )

                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()
                SOURCE_CROSS_ATTENTION_MASK_memmap[list_index] = tokenized_inputs["cross_attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap
                del SOURCE_CROSS_ATTENTION_MASK_memmap
                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

    if not os.path.isfile("processed/mhqg-wq/TEST_SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-wq/TEST_SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-wq/TEST_SOURCE_MASK_memmap.npy"
        SOURCE_CROSS_MASK_PATH = "processed/mhqg-wq/TEST_SOURCE_CROSS_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-wq/TEST_TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-wq/TEST_TARGET_MASK_memmap.npy"
        # input_length = 18624
        input_length = 2000
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        SOURCE_CROSS_MASK_memmap = np.memmap(
        filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del SOURCE_CROSS_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        with open("data/mhqg-wq/test.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += (x + " ")
                target = jo['outSeq']
                n = len(jo['inGraph']['g_node_names'])
                adj_matrix = {}
                adj_matrix2 = {}
                graph = {'node_name_id2word':{}}
                for idx, nid in enumerate(jo['inGraph']['g_node_names']):
                    # replace answer token to "<answer> answer <answer>"
                    if jo['inGraph']['g_node_names'][nid] in answers:
                        graph['node_name_id2word'][nid] = "<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"
                    else:
                        graph['node_name_id2word'][nid] = jo['inGraph']['g_node_names'][nid]

                    # create adj_matrix with 0 value
                    for idy, nid2 in enumerate(jo['inGraph']['g_node_names']):
                        if jo['inGraph']['g_node_names'][nid2] in answers:
                            adj_matrix2["<answer> "+ jo['inGraph']['g_node_names'][nid2] + " <answer>"] = 0
                        else:
                            adj_matrix2[jo['inGraph']['g_node_names'][nid2]] = 0
                    if jo['inGraph']['g_node_names'][nid] in answers:
                        adj_matrix["<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"] = adj_matrix2.copy()
                    else:
                        adj_matrix[jo['inGraph']['g_node_names'][nid]] = adj_matrix2.copy()
                for nid, val in jo['inGraph']['g_adj'].items():
                    for nid2, edge in val.items():
                        edge = edge.split('/')[-1]
                        adj_matrix[graph["node_name_id2word"][nid]][graph["node_name_id2word"][nid2]] = 1
                input_knowledge_graph = ""
                for i,ids in enumerate(list(adj_matrix.keys())):
                    input_knowledge_graph += (ids + " ")

                input, target = input_knowledge_graph, target

                tokenized_row_input = ""
                tokenized_row_input = tokenizer.tokenize(input)
                if len(tokenized_row_input) > 512:
                    continue

                tokenized_inputs = tokenizer.batch_encode_plus_self_attention(
                    [input_knowledge_graph], adjacancy_matrix=adj_matrix, max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )

                SOURCE_CROSS_ATTENTION_MASK_memmap = np.memmap(
                    filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512)
                )

                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()
                SOURCE_CROSS_ATTENTION_MASK_memmap[list_index] = tokenized_inputs["cross_attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap
                del SOURCE_CROSS_ATTENTION_MASK_memmap
                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

In [ ]:
def preprocess_pq(config):
    # preprocess input data
    if not os.path.isfile("processed/mhqg-pq/SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-pq/SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-pq/SOURCE_MASK_memmap.npy"
        SOURCE_CROSS_MASK_PATH = "processed/mhqg-pq/SOURCE_CROSS_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-pq/TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-pq/TARGET_MASK_memmap.npy"
        input_length = 9793
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        SOURCE_CROSS_MASK_memmap = np.memmap(
        filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del SOURCE_CROSS_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        with open("data/mhqg-pq/train.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += (x + " ")
                target = jo['outSeq']

                # used for model input
                adj_matrix = {}
                # used for create adjacency matrix
                adj_matrix2 = {}
                # map from id to node name
                graph = {'node_name_id2word':{}}

                # make graph mapping from id to node name
                for idx, nid in enumerate(jo['inGraph']['g_node_names']):
                    # replace answer token to "<answer> answer <answer>"
                    if jo['inGraph']['g_node_names'][nid] in answers:
                        graph['node_name_id2word'][nid] = "<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"
                    else:
                        graph['node_name_id2word'][nid] = jo['inGraph']['g_node_names'][nid]

                    # create adj_matrix with 0 value
                    for idy, nid2 in enumerate(jo['inGraph']['g_node_names']):
                        if jo['inGraph']['g_node_names'][nid2] in answers:
                            adj_matrix2["<answer> "+ jo['inGraph']['g_node_names'][nid2] + " <answer>"] = 0
                        else:
                            adj_matrix2[jo['inGraph']['g_node_names'][nid2]] = 0

                    if jo['inGraph']['g_node_names'][nid] in answers:
                        adj_matrix["<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"] = adj_matrix2.copy()
                    else:
                        adj_matrix[jo['inGraph']['g_node_names'][nid]] = adj_matrix2.copy()


                for nid, val in jo['inGraph']['g_adj'].items():
                    for nid2, edge in val.items():
                        adj_matrix[graph["node_name_id2word"][nid]][graph["node_name_id2word"][nid2]] = 1
                input_knowledge_graph = ""
                for i,ids in enumerate(list(adj_matrix.keys())):
                    input_knowledge_graph += (ids + " ")

                input, target = input_knowledge_graph, target

                tokenized_row_input = ""
                tokenized_row_input = tokenizer.tokenize(input)
                if len(tokenized_row_input) > 512:
                    continue

                tokenized_inputs = tokenizer.batch_encode_plus_self_attention(
                    [input_knowledge_graph], adjacancy_matrix=adj_matrix, max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )

                SOURCE_CROSS_ATTENTION_MASK_memmap = np.memmap(
                    filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512)
                )

                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()
                SOURCE_CROSS_ATTENTION_MASK_memmap[list_index] = tokenized_inputs["cross_attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap
                del SOURCE_CROSS_ATTENTION_MASK_memmap
                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

    # preprocess validation.json 
    if not os.path.isfile("processed/mhqg-pq/VAL_SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-pq/VAL_SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-pq/VAL_SOURCE_MASK_memmap.npy"
        SOURCE_CROSS_MASK_PATH = "processed/mhqg-pq/VAL_SOURCE_CROSS_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-pq/VAL_TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-pq/VAL_TARGET_MASK_memmap.npy"
        input_length = 1000
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        SOURCE_CROSS_MASK_memmap = np.memmap(
        filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del SOURCE_CROSS_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        with open("data/mhqg-pq/dev.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += (x + " ")
                target = jo['outSeq']

                # used for model input
                adj_matrix = {}
                # used for create adjacency matrix
                adj_matrix2 = {}
                # map from id to node name
                graph = {'node_name_id2word':{}}

                # make graph mapping from id to node name
                for idx, nid in enumerate(jo['inGraph']['g_node_names']):
                    # replace answer token to "<answer> answer <answer>"
                    if jo['inGraph']['g_node_names'][nid] in answers:
                        graph['node_name_id2word'][nid] = "<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"
                    else:
                        graph['node_name_id2word'][nid] = jo['inGraph']['g_node_names'][nid]

                    # create adj_matrix with 0 value
                    for idy, nid2 in enumerate(jo['inGraph']['g_node_names']):
                        if jo['inGraph']['g_node_names'][nid2] in answers:
                            adj_matrix2["<answer> "+ jo['inGraph']['g_node_names'][nid2] + " <answer>"] = 0
                        else:
                            adj_matrix2[jo['inGraph']['g_node_names'][nid2]] = 0

                    if jo['inGraph']['g_node_names'][nid] in answers:
                        adj_matrix["<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"] = adj_matrix2.copy()
                    else:
                        adj_matrix[jo['inGraph']['g_node_names'][nid]] = adj_matrix2.copy()


                for nid, val in jo['inGraph']['g_adj'].items():
                    for nid2, edge in val.items():
                        adj_matrix[graph["node_name_id2word"][nid]][graph["node_name_id2word"][nid2]] = 1
                input_knowledge_graph = ""
                for i,ids in enumerate(list(adj_matrix.keys())):
                    input_knowledge_graph += (ids + " ")

                input, target = input_knowledge_graph, target

                tokenized_row_input = ""
                tokenized_row_input = tokenizer.tokenize(input)
                if len(tokenized_row_input) > 512:
                    continue

                tokenized_inputs = tokenizer.batch_encode_plus_self_attention(
                    [input_knowledge_graph], adjacancy_matrix=adj_matrix, max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )

                SOURCE_CROSS_ATTENTION_MASK_memmap = np.memmap(
                    filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512)
                )

                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()
                SOURCE_CROSS_ATTENTION_MASK_memmap[list_index] = tokenized_inputs["cross_attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap
                del SOURCE_CROSS_ATTENTION_MASK_memmap
                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

    # preprocess test.json
    if not os.path.isfile("processed/mhqg-pq/TEST_SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-pq/TEST_SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-pq/TEST_SOURCE_MASK_memmap.npy"
        SOURCE_CROSS_MASK_PATH = "processed/mhqg-pq/TEST_SOURCE_CROSS_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-pq/TEST_TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-pq/TEST_TARGET_MASK_memmap.npy"
        # input_length = 18624
        input_length = 1000
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        SOURCE_CROSS_MASK_memmap = np.memmap(
        filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del SOURCE_CROSS_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        with open("data/mhqg-pq/test.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += (x + " ")
                target = jo['outSeq']

                # used for model input
                adj_matrix = {}
                # used for create adjacency matrix
                adj_matrix2 = {}
                # map from id to node name
                graph = {'node_name_id2word':{}}

                # make graph mapping from id to node name
                for idx, nid in enumerate(jo['inGraph']['g_node_names']):
                    # replace answer token to "<answer> answer <answer>"
                    if jo['inGraph']['g_node_names'][nid] in answers:
                        graph['node_name_id2word'][nid] = "<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"
                    else:
                        graph['node_name_id2word'][nid] = jo['inGraph']['g_node_names'][nid]

                    # create adj_matrix with 0 value
                    for idy, nid2 in enumerate(jo['inGraph']['g_node_names']):
                        if jo['inGraph']['g_node_names'][nid2] in answers:
                            adj_matrix2["<answer> "+ jo['inGraph']['g_node_names'][nid2] + " <answer>"] = 0
                        else:
                            adj_matrix2[jo['inGraph']['g_node_names'][nid2]] = 0

                    if jo['inGraph']['g_node_names'][nid] in answers:
                        adj_matrix["<answer> "+ jo['inGraph']['g_node_names'][nid] + " <answer>"] = adj_matrix2.copy()
                    else:
                        adj_matrix[jo['inGraph']['g_node_names'][nid]] = adj_matrix2.copy()


                for nid, val in jo['inGraph']['g_adj'].items():
                    for nid2, edge in val.items():
                        adj_matrix[graph["node_name_id2word"][nid]][graph["node_name_id2word"][nid2]] = 1
                input_knowledge_graph = ""
                for i,ids in enumerate(list(adj_matrix.keys())):
                    input_knowledge_graph += (ids + " ")

                input, target = input_knowledge_graph, target

                tokenized_row_input = ""
                tokenized_row_input = tokenizer.tokenize(input)
                if len(tokenized_row_input) > 512:
                    continue

                tokenized_inputs = tokenizer.batch_encode_plus_self_attention(
                    [input_knowledge_graph], adjacancy_matrix=adj_matrix, max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )

                SOURCE_CROSS_ATTENTION_MASK_memmap = np.memmap(
                    filename=SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512)
                )

                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()
                SOURCE_CROSS_ATTENTION_MASK_memmap[list_index] = tokenized_inputs["cross_attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap
                del SOURCE_CROSS_ATTENTION_MASK_memmap
                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

In [ ]:
class Preprocess():
    def __init__(self, config):
        if config.experiment.data=="mhqg-wq":
            preprocess_wq(config)
        elif config.experiment.data=="mhqg-pq":
            preprocess_pq(config)
        else:
            raise

## Dataset

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import json
import torch
from collections import Counter, defaultdict, OrderedDict
import gc
import os

class JsonDatasetWQ(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512, mode=None):
        assert mode=="Train" or mode=="Val" or mode=="Test"
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
 
        self.mode = mode
 
        if mode=="Train":
          self.SOURCE_ID_PATH = "processed/mhqg-wq/SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-wq/SOURCE_MASK_memmap.npy"
          self.SOURCE_CROSS_MASK_PATH = "processed/mhqg-wq/SOURCE_CROSS_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-wq/TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-wq/TARGET_MASK_memmap.npy"
        #   self.input_length = 18624
          self.input_length = 18989
          self.list_index = 18989
        elif mode=="Val":
          self.SOURCE_ID_PATH = "processed/mhqg-wq/VAL_SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-wq/VAL_SOURCE_MASK_memmap.npy"
          self.SOURCE_CROSS_MASK_PATH = "processed/mhqg-wq/VAL_SOURCE_CROSS_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-wq/VAL_TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-wq/VAL_TARGET_MASK_memmap.npy"
          self.input_length = 2000
          self.list_index = 2000
        #   self.input_length = 1985
        elif mode=="Test":
          self.SOURCE_ID_PATH = "processed/mhqg-wq/TEST_SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-wq/TEST_SOURCE_MASK_memmap.npy"
          self.SOURCE_CROSS_MASK_PATH = "processed/mhqg-wq/TEST_SOURCE_CROSS_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-wq/TEST_TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-wq/TEST_TARGET_MASK_memmap.npy"
          self.input_length = 2000
          self.list_index = 2000
          
        #   self.input_length = 1985
 
        self.SOURCE_ID_memmap = np.memmap(
          filename=self.SOURCE_ID_PATH, dtype=np.int64, mode="r",shape=(self.input_length,512) 
        )
 
        self.SOURCE_MASK_memmap = np.memmap(
          filename=self.SOURCE_MASK_PATH, dtype=np.int64, mode="r", shape=(self.input_length,512,512)
        )

        self.VAL_SOURCE_CROSS_MASK_memmap = np.memmap(
          filename=self.SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="r", shape=(self.input_length,512)
        )
 
        self.TARGET_ID_memmap = np.memmap(
          filename=self.TARGET_ID_PATH, dtype=np.int64, mode="r", shape=(self.input_length,100)
        )
 
        self.TARGET_MASK_memmap = np.memmap(
          filename=self.TARGET_MASK_PATH, dtype=np.int64, mode="r",shape=(self.input_length,100)
        )

  
    def __len__(self):
        return self.list_index
  
    def __getitem__(self, index):
        return {"source_ids": torch.from_numpy(np.array(self.SOURCE_ID_memmap[index])).squeeze(), "source_mask": torch.from_numpy(np.array(self.SOURCE_MASK_memmap[index])).squeeze(), "cross_attention_mask":torch.from_numpy(np.array(self.VAL_SOURCE_CROSS_MASK_memmap[index])).squeeze(),
                "target_ids": torch.from_numpy(np.array(self.TARGET_ID_memmap[index])).squeeze(), "target_mask": torch.from_numpy(np.array(self.TARGET_MASK_memmap[index])).squeeze()}

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import json
import torch
from collections import Counter, defaultdict, OrderedDict
import gc
import os

class JsonDatasetPQ(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512, mode=None):
        assert mode=="Train" or mode=="Val" or mode=="Test"
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
 
        self.mode = mode
 
        if mode=="Train":
          self.SOURCE_ID_PATH = "processed/mhqg-pq/SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-pq/SOURCE_MASK_memmap.npy"
          self.SOURCE_CROSS_MASK_PATH = "processed/mhqg-pq/SOURCE_CROSS_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-pq/TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-pq/TARGET_MASK_memmap.npy"
          self.input_length = 9793
          self.list_index = 9793
        elif mode=="Val":
          self.SOURCE_ID_PATH = "processed/mhqg-pq/VAL_SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-pq/VAL_SOURCE_MASK_memmap.npy"
          self.SOURCE_CROSS_MASK_PATH = "processed/mhqg-pq/VAL_SOURCE_CROSS_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-pq/VAL_TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-pq/VAL_TARGET_MASK_memmap.npy"
          self.input_length = 1000
          self.list_index = 1000
        elif mode=="Test":
          self.SOURCE_ID_PATH = "processed/mhqg-pq/TEST_SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-pq/TEST_SOURCE_MASK_memmap.npy"
          self.SOURCE_CROSS_MASK_PATH = "processed/mhqg-pq/TEST_SOURCE_CROSS_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-pq/TEST_TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-pq/TEST_TARGET_MASK_memmap.npy"
          self.input_length = 1000
          self.list_index = 1000
          
 
        self.SOURCE_ID_memmap = np.memmap(
          filename=self.SOURCE_ID_PATH, dtype=np.int64, mode="r",shape=(self.input_length,512) 
        )
 
        self.SOURCE_MASK_memmap = np.memmap(
          filename=self.SOURCE_MASK_PATH, dtype=np.int64, mode="r", shape=(self.input_length,512,512)
        )

        self.VAL_SOURCE_CROSS_MASK_memmap = np.memmap(
          filename=self.SOURCE_CROSS_MASK_PATH, dtype=np.int64, mode="r", shape=(self.input_length,512)
        )
 
        self.TARGET_ID_memmap = np.memmap(
          filename=self.TARGET_ID_PATH, dtype=np.int64, mode="r", shape=(self.input_length,100)
        )
 
        self.TARGET_MASK_memmap = np.memmap(
          filename=self.TARGET_MASK_PATH, dtype=np.int64, mode="r",shape=(self.input_length,100)
        )

  
    def __len__(self):
        return self.list_index
  
    def __getitem__(self, index):
        return {"source_ids": torch.from_numpy(np.array(self.SOURCE_ID_memmap[index])).squeeze(), "source_mask": torch.from_numpy(np.array(self.SOURCE_MASK_memmap[index])).squeeze(), "cross_attention_mask":torch.from_numpy(np.array(self.VAL_SOURCE_CROSS_MASK_memmap[index])).squeeze(),
                "target_ids": torch.from_numpy(np.array(self.TARGET_ID_memmap[index])).squeeze(), "target_mask": torch.from_numpy(np.array(self.TARGET_MASK_memmap[index])).squeeze()}

## Training

In [ ]:
import pytorch_lightning as pl
from AdjacencyAttentionWithoutSelfloopTransformers import (
  AdamW,
  T5ForConditionalGeneration,
  T5Tokenizer,
  get_linear_schedule_with_warmup
)
from torch.utils.data import DataLoader

from tqdm.auto import tqdm

import pandas as pd
from core.evaluation.eval import QGEvalCap

from omegaconf import DictConfig

def saveOutputs(hparams: DictConfig, inputs: list, outputs: list, targets: list) -> None:
  data = pd.DataFrame(list(zip(inputs, outputs, targets)), columns =['inputs', 'outputs', 'targets'])
  data.to_csv(os.path.join("out",hparams.experiment.model_dir.split("/")[-1]+".csv"),index=False, header=True)

def run_eval(target_src, decoded_text) -> dict:
  assert len(target_src) == len(decoded_text)
  eval_targets = {}
  eval_predictions = {}
  for idx in range(len(target_src)):
      eval_targets[idx] = [target_src[idx]]
      eval_predictions[idx] = [decoded_text[idx]]

  QGEval = QGEvalCap(eval_targets, eval_predictions)
  scores = QGEval.evaluate()
  return scores

class T5FineTuner(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(config.experiment.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(config.experiment.tokenizer_name_or_path, is_fast=True)

        special_tokens_dict = {'sep_token': '<sep>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask=None, cross_attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None, mode=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            cross_attention_mask=cross_attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
            three_dim_attention_mask=True
        )

    def _step(self, batch, mode=None):
        """ロス計算"""
        labels = batch["target_ids"].detach().clone()

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            cross_attention_mask=batch["cross_attention_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels,
            mode=mode
        )

        return outputs

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        outputs = self._step(batch)
        loss = outputs[0]
        logit = outputs[1]
        # logging metrics we calculated by hand
        self.log('train/loss', loss, on_epoch=True)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        outputs = self._step(batch)
        loss = outputs[0]
        logit = outputs[1]
        self.log("valid/loss_epoch", loss)  # default on val/test is on_epoch only
        # return {"val_loss": loss}

    def test_epoch_end(self, training_step_outputs):
        saveOutputs(self.config, self.inputs, self.outputs, self.targets)
        scores = run_eval(self.targets, self.outputs)
        wandb.log(scores)

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.config.training.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.config.training.learning_rate, 
                          eps=self.config.training.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.config.training.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args, mode=None):
        """データセットを作成する"""
        if args.experiment.data=="mhqg-wq":
          return JsonDatasetWQ(
            tokenizer=tokenizer, 
            data_dir=args.experiment.data_dir, 
            type_path=type_path, 
            input_max_len=args.model.max_input_length,
            target_max_len=args.model.max_target_length,
            mode=mode)
        else:
          return JsonDatasetPQ(
            tokenizer=tokenizer, 
            data_dir=args.experiment.data_dir, 
            type_path=type_path, 
            input_max_len=args.model.max_input_length,
            target_max_len=args.model.max_target_length,
            mode=mode)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.json", args=self.config, mode="Train")
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="dev.json", args=self.config, mode="Val")
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.config.training.train_batch_size * max(1, self.config.training.n_gpu)))
                // self.config.training.gradient_accumulation_steps
                * float(self.config.training.num_train_epochs)
            )
        elif stage == 'test':
            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="test.json", args=self.config, mode="Test")
            self.test_dataset = test_dataset

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.config.training.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.config.training.eval_batch_size, 
                          num_workers=4)
        
    def test_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.config.training.test_batch_size, 
                          num_workers=4)

## Evaluation

In [ ]:
from AdjacencyAttentionWithoutSelfloopTransformers import (
  AdamW,
  T5ForConditionalGeneration,
  T5Tokenizer,
  get_linear_schedule_with_warmup
)
from torch.utils.data import DataLoader

from tqdm.auto import tqdm

import pandas as pd
from core.evaluation.eval import QGEvalCap

from omegaconf import DictConfig


# OmegaConf.register_new_resolver("now", lambda pattern: strftime(pattern, localtime()))

def saveOutputs(hparams: DictConfig, inputs: list, outputs: list, targets: list) -> None:
  data = pd.DataFrame(list(zip(inputs, outputs, targets)), columns =['inputs', 'outputs', 'targets'])
  data.to_csv(os.path.join("out",hparams.experiment.model_dir.split("/")[-1]+".csv"),index=False, header=True)

def run_eval(target_src, decoded_text) -> dict:
  assert len(target_src) == len(decoded_text)
  eval_targets = {}
  eval_predictions = {}
  for idx in range(len(target_src)):
      eval_targets[idx] = [target_src[idx]]
      eval_predictions[idx] = [decoded_text[idx]]

  QGEval = QGEvalCap(eval_targets, eval_predictions)
  scores = QGEval.evaluate()
  return scores

class T5Evaluation(pl.LightningModule):
    def __init__(self, config: DictConfig, train_params: DictConfig):
        super().__init__()
        self.config = config
        self.train_params = train_params
        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(self.config.experiment.model_dir)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(self.config.experiment.model_dir, is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        self.save_hyperparameters()

        self.inputs = []
        self.outputs = []
        self.targets = []


    def forward(self, input_ids, attention_mask=None, cross_attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model.generate(input_ids=input_ids, 
            attention_mask=attention_mask, 
            cross_attention_mask=cross_attention_mask,
            max_length=self.config.model.max_target_length,
            temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
            repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
            three_dim_attention_mask=True
            )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            cross_attention_mask=batch["cross_attention_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        return outputs

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        output = self._step(batch)
        labels = batch["target_ids"]
        labels[labels[:, :] == -100] = 0
        output_text = [self.tokenizer.decode(ids, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=False) 
                    for ids in output]
        target_text = [self.tokenizer.decode(ids, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=False) 
                    for ids in labels]
        input_text = [self.tokenizer.decode(ids, skip_special_tokens=False, 
                                clean_up_tokenization_spaces=False) 
                    for ids in batch["source_ids"]]

        self.inputs.extend(input_text)
        self.outputs.extend(output_text)
        self.targets.extend(target_text)

        return {"batch_idx":batch_idx}
        
    
    def test_epoch_end(self, training_step_outputs):
        saveOutputs(self.config, self.inputs, self.outputs, self.targets)
        scores = run_eval(self.targets, self.outputs)
        wandb.log(scores)

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.config.training.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.config.training.learning_rate, 
                          eps=self.config.training.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.config.training.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, mode=None):
        """データセットを作成する"""
        if self.config.get('experiment').data=="mhqg-wq":
          return JsonDatasetWQ(
            tokenizer=tokenizer, 
            data_dir=self.config.experiment.data_dir, 
            type_path=type_path, 
            input_max_len=self.config.model.max_input_length,
            target_max_len=self.config.model.max_target_length,
            mode=mode)
        else:
          return JsonDatasetPQ(
            tokenizer=tokenizer, 
            data_dir=self.config.experiment.data_dir, 
            type_path=type_path, 
            input_max_len=self.config.model.max_input_length,
            target_max_len=self.config.model.max_target_length,
            mode=mode)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'test':
            test_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="test.json", mode="Test")
            self.test_dataset = test_dataset
        
    def test_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.test_dataset, 
                          batch_size=self.config.training.test_batch_size, 
                          num_workers=4)

class Evaluation():
  def __init__(self, config, train_params):
    self.config = config
    self.train_params = train_params

  def run(self):
    # Tokenizer
    tokenizer = T5Tokenizer.from_pretrained(self.config.experiment.model_dir, is_fast=True)
    trained_model = T5ForConditionalGeneration.from_pretrained(self.config.experiment.model_dir)

    if self.config.experiment.data=="mhqg-wq":
        # import test data
        test_dataset = JsonDatasetWQ(tokenizer, self.config.experiment.data_dir, "test.json", 
                                input_max_len=self.config.model.max_input_length, 
                                target_max_len=self.config.model.max_target_length,
                                mode="Test")

    test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

    trained_model.eval()

    inputs = []
    outputs = []
    targets = []

    for index, batch in enumerate(tqdm(test_loader)):
        input_ids = batch['source_ids']
        input_mask = batch['source_mask']
        input_cross_attention_mask = batch["cross_attention_mask"]
        if self.config.training.n_gpu:
            input_ids = input_ids.cuda()
            input_mask = input_mask.cuda()

        output = trained_model.generate(input_ids=input_ids, 
            attention_mask=input_mask, 
            cross_attention_mask=input_cross_attention_mask,
            max_length=self.config.model.max_target_length,
            temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
            repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
            three_dim_attention_mask=True
            )

        output_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=False) 
                    for ids in output]
        target_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=False) 
                    for ids in batch["target_ids"]]
        input_text = [tokenizer.decode(ids, skip_special_tokens=False, 
                                clean_up_tokenization_spaces=False) 
                    for ids in input_ids]

        inputs.extend(input_text)
        outputs.extend(output_text)
        targets.extend(target_text)

        saveOutputs(self.config, inputs, outputs, targets)
        scores = run_eval(targets, outputs)
        self.train_params["logger"].log_metrics(scores)

## Main

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
from itertools import chain
from string import punctuation
 
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from collections import Counter, defaultdict, OrderedDict
from nltk.tokenize import wordpunct_tokenize, word_tokenize
 
import gc

from omegaconf import DictConfig
import omegaconf

import hydra
import pytorch_lightning as pl



import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_train_param(config: DictConfig) -> dict:
    wandb_logger = WandbLogger(
        name=("exp_" + str(config.experiment.wandb.exp_num)),
        project=config.experiment.wandb.project,
        log_model=True,
    )
    checkpoint_path = os.path.join(
        config.experiment.model_dir, config.experiment.wandb.checkpoint_path
    )
    wandb_logger.log_hyperparams(config)
    early_stopping = EarlyStopping('valid/loss_epoch')
    train_params = dict(
        callbacks=[early_stopping],
        logger = wandb_logger,
        accumulate_grad_batches=config.training.gradient_accumulation_steps,
        gpus=config.training.n_gpu,
        max_epochs=config.training.num_train_epochs,
        precision= 16 if config.training.fp_16 else 32,
        amp_level=config.training.opt_level,
        gradient_clip_val=config.training.max_grad_norm,
    )
    return train_params

# @hydra.main(config_path="config.yaml")
def train(config: DictConfig, train_params: dict) -> None:
    
    # conduct transfer learning
    model = T5FineTuner(config)
    wandb.watch(model, log_freq=100)
    trainer = pl.Trainer(**train_params)
    trainer.fit(model)

    model.tokenizer.save_pretrained(config.experiment.model_dir)
    model.model.save_pretrained(config.experiment.model_dir)

    # trainer.test(model)

    # torch.save(
    #             {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth"
    #         )

# @hydra.main(config_path="adjacencyattentionwithoutselfloop", config_name="config")
def evaluation(config: DictConfig, train_params: dict) -> None:
    print("config",config)
    print("type_config",type(config))
    set_seed(42)
    USE_GPU = torch.cuda.is_available()
    config.training.n_gpu=1 if USE_GPU else 0
    eval_model = T5Evaluation(config, train_params)
    trainer = pl.Trainer(**train_params)
    trainer.test(eval_model)
  
def main(config: DictConfig) -> None:
    if os.path.exists(config.experiment.model_dir):
        raise "model_dir is exist"

    set_seed(42)
    USE_GPU = torch.cuda.is_available()
    config.training.n_gpu=1 if USE_GPU else 0
    train_params = get_train_param(config)

    Preprocess(config)
    # train(config, train_params)
    evaluation(config, train_params)


In [ ]:
config = omegaconf.OmegaConf.load("config/config_test_pq.yaml")
main(config)
wandb.finish()

TypeError: ignored

In [ ]:
config = omegaconf.OmegaConf.load("config/config_test_pq.yaml")
train_params = get_train_param(config)
evaluation(config, train_params)

config {'experiment': {'seed': 42, 'data': 'mhqg-pq', 'data_dir': 'data/mhqg-pq/', 'model_name_or_path': 't5-small', 'tokenizer_name_or_path': 't5-small', 'model_dir': 'content/test_mhqg-pq_t5-small_20210905_1310', 'wandb': {'exp_num': 5, 'project': 'test_mhqg-pq_t5-small', 'checkpoint_path': 'checkpoint/'}}, 'model': {'max_input_length': 100, 'max_target_length': 100}, 'training': {'learning_rate': 0.0003, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'gradient_accumulation_steps': 1, 'early_stop_callback': True, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42, 'train_batch_size': 32, 'eval_batch_size': 32, 'test_batch_size': 8, 'num_train_epochs': 10, 'precision': 16, 'n_gpu': 1}}
type_config <class 'omegaconf.dictconfig.DictConfig'>


GPU available: True, used: True
TPU available: None, using: 0 TPU cores


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


# 20210918

In [1]:
cd /content/drive/MyDrive/ColabNotebooks/Research/KG2QGwithT5/TripleAttention

/content/drive/MyDrive/ColabNotebooks/Research/KG2QGwithT5/TripleAttention


In [ ]:
!pip install -q --pre poetry==1.1.8

     |████████████████████████████████| 173 kB 31.6 MB/s 
     |████████████████████████████████| 5.3 MB 33.9 MB/s 
     |████████████████████████████████| 91 kB 9.0 MB/s 
     |████████████████████████████████| 420 kB 68.3 MB/s 
     |████████████████████████████████| 40 kB 5.5 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 3.0 MB 66.8 MB/s 
     |████████████████████████████████| 338 kB 77.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## Install dependency
~~~
Based on the pyproject.toml in the current path, dependency will be installed in order.
~~~

In [ ]:
!pip install --requirement <(poetry export --format requirements.txt)

Ignoring colorama: markers 'python_version >= "3.6" and python_full_version < "3.0.0" and platform_system == "Windows" or python_full_version >= "3.5.0" and python_version >= "3.6" and platform_system == "Windows"' don't match your environment
Ignoring pywin32: markers 'sys_platform == "win32" and python_version >= "3.6"' don't match your environment
Ignoring waitress: markers 'platform_system == "Windows" and python_version >= "3.6" and python_full_version >= "3.6.0"' don't match your environment
     |████████████████████████████████| 132 kB 30.6 MB/s 
     |████████████████████████████████| 1.3 MB 42.8 MB/s 
     |████████████████████████████████| 1.1 MB 46.1 MB/s 
     |████████████████████████████████| 112 kB 58.4 MB/s 
     |████████████████████████████████| 178 kB 58.4 MB/s 
     |████████████████████████████████| 97 kB 7.5 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 145 kB 59.4 MB/s 
     |███████████████████████████████

## Import Library

In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
 
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

import io
import string
from collections import Counter, defaultdict, OrderedDict
from nltk.tokenize import wordpunct_tokenize, word_tokenize
 
import gc

from mytransformers.src.transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

import os.path

import wandb
wandb.login()

wandb: Currently logged in as: macho000 (use `wandb login --relogin` to force relogin)


True

## Preprocess

In [15]:
def preprocess_wq(config):
    if not os.path.isfile("processed/mhqg-wq/SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-wq/SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-wq/SOURCE_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-wq/TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-wq/TARGET_MASK_memmap.npy"
        # input_length = 18624
        input_length = 18989
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        def _make_record(answer, input, target):
            # 質問生成タスク用の入出力形式に変換する。
            input = f"{answer}{input}"
            target = f"{target}"
            return input, target

        with open("data/mhqg-wq/train.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += " <answer> "
                    normalized_answers += x
                target = jo['outSeq']

                graph = {'node_name_id2word':{}}
                for idx, nid in enumerate(jo['inGraph']['g_node_names']):
                    graph['node_name_id2word'][nid] = jo['inGraph']['g_node_names'][nid]
                input_knowledge_graph = ""
                for nid, val in jo['inGraph']['g_adj'].items():
                    for nid2, edge in val.items():
                        edge = edge.split('/')[-1]
                        input_knowledge_graph += " <sep> " + " <subject> "+ str(graph["node_name_id2word"][nid])+" <relation> "+str(edge)+" <object> "+str(graph["node_name_id2word"][nid2])
                
                input, target = _make_record(normalized_answers,input_knowledge_graph, target)
 
                tokenized_inputs = tokenizer.batch_encode_plus(
                    [input], max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
 
                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )


                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap

                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

    if not os.path.isfile("processed/mhqg-wq/VAL_SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-wq/VAL_SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-wq/VAL_SOURCE_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-wq/VAL_TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-wq/VAL_TARGET_MASK_memmap.npy"
        # input_length = 18624
        input_length = 2000
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        def _make_record(answer, input, target):
            # 質問生成タスク用の入出力形式に変換する。
            input = f"{answer}{input}"
            target = f"{target}"
            return input, target

        with open("data/mhqg-wq/dev.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += " <answer> "
                    normalized_answers += x
                target = jo['outSeq']

                graph = {'node_name_id2word':{}}
                for idx, nid in enumerate(jo['inGraph']['g_node_names']):
                    graph['node_name_id2word'][nid] = jo['inGraph']['g_node_names'][nid]
                input_knowledge_graph = ""
                for nid, val in jo['inGraph']['g_adj'].items():
                    for nid2, edge in val.items():
                        edge = edge.split('/')[-1]
                        input_knowledge_graph += " <sep> " + " <subject> "+ str(graph["node_name_id2word"][nid])+" <relation> "+str(edge)+" <object> "+str(graph["node_name_id2word"][nid2])
                
                input, target = _make_record(normalized_answers,input_knowledge_graph, target)
 
                tokenized_inputs = tokenizer.batch_encode_plus(
                    [input], max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
 
                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )


                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap

                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

    if not os.path.isfile("processed/mhqg-wq/TEST_SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-wq/TEST_SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-wq/TEST_SOURCE_MASK_memmap.npy"
        SOURCE_CROSS_MASK_PATH = "processed/mhqg-wq/TEST_SOURCE_CROSS_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-wq/TEST_TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-wq/TEST_TARGET_MASK_memmap.npy"
        # input_length = 18624
        input_length = 2000
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        def _make_record(answer, input, target):
            # 質問生成タスク用の入出力形式に変換する。
            input = f"{answer}{input}"
            target = f"{target}"
            return input, target

        with open("data/mhqg-wq/test.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += " <answer> "
                    normalized_answers += x
                target = jo['outSeq']
                
                graph = {'node_name_id2word':{}}
                for idx, nid in enumerate(jo['inGraph']['g_node_names']):
                    graph['node_name_id2word'][nid] = jo['inGraph']['g_node_names'][nid]
                input_knowledge_graph = ""
                for nid, val in jo['inGraph']['g_adj'].items():
                    for nid2, edge in val.items():
                        edge = edge.split('/')[-1]
                        input_knowledge_graph += " <sep> " + " <subject> "+ str(graph["node_name_id2word"][nid])+" <relation> "+str(edge)+" <object> "+str(graph["node_name_id2word"][nid2])
                
                input, target = _make_record(normalized_answers,input_knowledge_graph, target)
 
                tokenized_inputs = tokenizer.batch_encode_plus(
                    [input], max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
 
                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )


                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap

                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

In [4]:
def preprocess_pq(config):
    # preprocess input data
    if not os.path.isfile("processed/mhqg-pq/SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-pq/SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-pq/SOURCE_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-pq/TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-pq/TARGET_MASK_memmap.npy"
        input_length = 9793
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        def _make_record(answer, input, target):
            #   質問生成タスク用の入出力形式に変換する。
            input = f"{answer}{input}"
            target = f"{target}"
            return input, target

        with open("data/mhqg-pq/train.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += " <answer> "
                    normalized_answers += x
                target = jo['outSeq']

                kglist = []
                for node in jo["inGraph"]["g_node_names"]:
                  try:
                    kglist.append((node,jo["inGraph"]["g_adj"][node]))
                  except KeyError:
                    continue

                input_knowledge_graph = ""
                for subject, dict_of_subject in kglist:
                  for node in jo["inGraph"]["g_node_names"]:
                    try:
                      input_knowledge_graph += " <sep> " + " <subject> "+ str(jo["inGraph"]["g_node_names"][subject])+" <relation> "+str(" ".join(dict_of_subject[node][0].split('/')[-1].split('_')))+" <object> "+str(jo["inGraph"]["g_node_names"][node])
                    except KeyError:
                      continue
                
                input, target = _make_record(normalized_answers,input_knowledge_graph, target)

                tokenized_inputs = tokenizer.batch_encode_plus(
                    [input], max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_row_input = ""
                tokenized_row_input = tokenizer.tokenize(input)
                if len(tokenized_row_input) > 512:
                  continue

                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )

                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap
                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

    # preprocess validation.json 
    if not os.path.isfile("processed/mhqg-pq/VAL_SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-pq/VAL_SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-pq/VAL_SOURCE_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-pq/VAL_TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-pq/VAL_TARGET_MASK_memmap.npy"
        input_length = 1000
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        def _make_record(answer, input, target):
            # 質問生成タスク用の入出力形式に変換する。
            input = f"{answer}{input}"
            target = f"{target}"
            return input, target

        with open("data/mhqg-pq/dev.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += " <answer> "
                    normalized_answers += x
                target = jo['outSeq']

                kglist = []
                for node in jo["inGraph"]["g_node_names"]:
                  try:
                    kglist.append((node,jo["inGraph"]["g_adj"][node]))
                  except KeyError:
                    continue

                input_knowledge_graph = ""
                for subject, dict_of_subject in kglist:
                  for node in jo["inGraph"]["g_node_names"]:
                    try:
                      input_knowledge_graph += " <sep> " + " <subject> "+ str(jo["inGraph"]["g_node_names"][subject])+" <relation> "+str(" ".join(dict_of_subject[node][0].split('/')[-1].split('_')))+" <object> "+str(jo["inGraph"]["g_node_names"][node])
                    except KeyError:
                      continue
                
                input, target = _make_record(normalized_answers,input_knowledge_graph, target)

                tokenized_inputs = tokenizer.batch_encode_plus(
                    [input], max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_row_input = ""
                tokenized_row_input = tokenizer.tokenize(input)
                if len(tokenized_row_input) > 512:
                  continue

                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )

                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap
                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

    # preprocess test.json
    if not os.path.isfile("processed/mhqg-pq/TEST_SOURCE_ID_memmap.npy"):
        SOURCE_ID_PATH = "processed/mhqg-pq/TEST_SOURCE_ID_memmap.npy"
        SOURCE_MASK_PATH = "processed/mhqg-pq/TEST_SOURCE_MASK_memmap.npy"
        TARGET_ID_PATH = "processed/mhqg-pq/TEST_TARGET_ID_memmap.npy"
        TARGET_MASK_PATH = "processed/mhqg-pq/TEST_TARGET_MASK_memmap.npy"
        # input_length = 18624
        input_length = 1000
        SOURCE_ID_memmap = np.memmap(
        filename=SOURCE_ID_PATH, dtype=np.int64, mode="w+",shape=(input_length,512) 
        )

        SOURCE_MASK_memmap = np.memmap(
        filename=SOURCE_MASK_PATH, dtype=np.int64, mode="w+", shape=(input_length,512,512)
        )

        TARGET_ID_memmap = np.memmap(
        filename=TARGET_ID_PATH, dtype=np.int64, mode="w+", shape=(input_length,100)
        )

        TARGET_MASK_memmap = np.memmap(
        filename=TARGET_MASK_PATH, dtype=np.int64, mode="w+",shape=(input_length,100)
        )

        del SOURCE_ID_memmap
        del SOURCE_MASK_memmap
        del TARGET_ID_memmap
        del TARGET_MASK_memmap
        tokenizer = T5Tokenizer.from_pretrained("t5-small", is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        tokenizer.add_special_tokens(special_tokens_dict)

        def _make_record(answer, input, target):
            # 質問生成タスク用の入出力形式に変換する。
            input = f"{answer}{input}"
            target = f"{target}"
            return input, target

        with open("data/mhqg-pq/test.json", 'r') as f:
            list_index = 0
            for index, line in enumerate(f):
                line = line.strip()
                jo = json.loads(line, object_pairs_hook=OrderedDict)
                assert len(jo['inGraph']['g_adj']) > 0

                answers = jo['answers']
                normalized_answers = ""
                for x in answers:
                    normalized_answers += " <answer> "
                    normalized_answers += x
                target = jo['outSeq']

                kglist = []
                for node in jo["inGraph"]["g_node_names"]:
                  try:
                    kglist.append((node,jo["inGraph"]["g_adj"][node]))
                  except KeyError:
                    continue

                input_knowledge_graph = ""
                for subject, dict_of_subject in kglist:
                  for node in jo["inGraph"]["g_node_names"]:
                    try:
                      input_knowledge_graph += " <sep> " + " <subject> "+ str(jo["inGraph"]["g_node_names"][subject])+" <relation> "+str(" ".join(dict_of_subject[node][0].split('/')[-1].split('_')))+" <object> "+str(jo["inGraph"]["g_node_names"][node])
                    except KeyError:
                      continue
                
                input, target = _make_record(normalized_answers,input_knowledge_graph, target)

                tokenized_inputs = tokenizer.batch_encode_plus(
                    [input], max_length=512, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = tokenizer.batch_encode_plus(
                    [target], max_length=100, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_row_input = ""
                tokenized_row_input = tokenizer.tokenize(input)
                if len(tokenized_row_input) > 512:
                  continue

                SOURCE_ID_memmap = np.memmap(
                    filename=SOURCE_ID_PATH, dtype=np.int64, mode="r+",shape=(input_length,512) 
                )

                SOURCE_MASK_memmap = np.memmap(
                    filename=SOURCE_MASK_PATH, dtype=np.int64, mode="r+", shape=(input_length,512,512)
                )

                TARGET_ID_memmap = np.memmap(
                    filename=TARGET_ID_PATH, dtype=np.int64, mode="r+", shape=(input_length,100)
                )

                TARGET_MASK_memmap = np.memmap(
                    filename=TARGET_MASK_PATH, dtype=np.int64, mode="r+",shape=(input_length,100)
                )

                SOURCE_ID_memmap[list_index] = tokenized_inputs["input_ids"].squeeze().numpy()
                SOURCE_MASK_memmap[list_index] = tokenized_inputs["attention_mask"].squeeze().numpy()

                TARGET_ID_memmap[list_index] = tokenized_targets["input_ids"].squeeze().numpy()
                TARGET_MASK_memmap[list_index] = tokenized_targets["attention_mask"].squeeze().numpy()

                del SOURCE_ID_memmap
                del SOURCE_MASK_memmap
                del TARGET_ID_memmap
                del TARGET_MASK_memmap

                if index%500==0:
                    print(f"{index} was finished", index)

                list_index += 1

In [5]:
class Preprocess():
    def __init__(self, config):
        if config.experiment.data=="mhqg-wq":
            preprocess_wq(config)
        elif config.experiment.data=="mhqg-pq":
            preprocess_pq(config)
        else:
            raise

## Dataset

In [18]:
from torch.utils.data import Dataset
import numpy as np
import json
import torch
from collections import Counter, defaultdict, OrderedDict
import gc
import os

class JsonDatasetWQ(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512, mode=None):
        assert mode=="Train" or mode=="Val" or mode=="Test"
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
 
        self.mode = mode
 
        if mode=="Train":
          self.SOURCE_ID_PATH = "processed/mhqg-wq/SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-wq/SOURCE_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-wq/TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-wq/TARGET_MASK_memmap.npy"
        #   self.input_length = 18624
          self.input_length = 18989
          self.list_index = 18989
        elif mode=="Val":
          self.SOURCE_ID_PATH = "processed/mhqg-wq/VAL_SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-wq/VAL_SOURCE_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-wq/VAL_TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-wq/VAL_TARGET_MASK_memmap.npy"
          self.input_length = 2000
          self.list_index = 2000
        #   self.input_length = 1985
        elif mode=="Test":
          self.SOURCE_ID_PATH = "processed/mhqg-wq/TEST_SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-wq/TEST_SOURCE_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-wq/TEST_TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-wq/TEST_TARGET_MASK_memmap.npy"
          self.input_length = 2000
          self.list_index = 2000
          
        #   self.input_length = 1985
 
        self.SOURCE_ID_memmap = np.memmap(
          filename=self.SOURCE_ID_PATH, dtype=np.int64, mode="r",shape=(self.input_length,512) 
        )
 
        self.SOURCE_MASK_memmap = np.memmap(
          filename=self.SOURCE_MASK_PATH, dtype=np.int64, mode="r", shape=(self.input_length,512,512)
        )

        self.TARGET_ID_memmap = np.memmap(
          filename=self.TARGET_ID_PATH, dtype=np.int64, mode="r", shape=(self.input_length,100)
        )
 
        self.TARGET_MASK_memmap = np.memmap(
          filename=self.TARGET_MASK_PATH, dtype=np.int64, mode="r",shape=(self.input_length,100)
        )

  
    def __len__(self):
        return self.list_index
  
    def __getitem__(self, index):
        return {"source_ids": torch.from_numpy(np.array(self.SOURCE_ID_memmap[index])).squeeze(), "source_mask": torch.from_numpy(np.array(self.SOURCE_MASK_memmap[index])).squeeze(),
                "target_ids": torch.from_numpy(np.array(self.TARGET_ID_memmap[index])).squeeze(), "target_mask": torch.from_numpy(np.array(self.TARGET_MASK_memmap[index])).squeeze()}

In [7]:
from torch.utils.data import Dataset
import numpy as np
import json
import torch
from collections import Counter, defaultdict, OrderedDict
import gc
import os

class JsonDatasetPQ(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512, mode=None):
        assert mode=="Train" or mode=="Val" or mode=="Test"
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
 
        self.mode = mode
 
        if mode=="Train":
          self.SOURCE_ID_PATH = "processed/mhqg-pq/SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-pq/SOURCE_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-pq/TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-pq/TARGET_MASK_memmap.npy"
          self.input_length = 9793
          self.list_index = 9793
        elif mode=="Val":
          self.SOURCE_ID_PATH = "processed/mhqg-pq/VAL_SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-pq/VAL_SOURCE_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-pq/VAL_TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-pq/VAL_TARGET_MASK_memmap.npy"
          self.input_length = 1000
          self.list_index = 1000
        elif mode=="Test":
          self.SOURCE_ID_PATH = "processed/mhqg-pq/TEST_SOURCE_ID_memmap.npy"
          self.SOURCE_MASK_PATH = "processed/mhqg-pq/TEST_SOURCE_MASK_memmap.npy"
          self.TARGET_ID_PATH = "processed/mhqg-pq/TEST_TARGET_ID_memmap.npy"
          self.TARGET_MASK_PATH = "processed/mhqg-pq/TEST_TARGET_MASK_memmap.npy"
          self.input_length = 1000
          self.list_index = 1000
          
 
        self.SOURCE_ID_memmap = np.memmap(
          filename=self.SOURCE_ID_PATH, dtype=np.int64, mode="r",shape=(self.input_length,512) 
        )
 
        self.SOURCE_MASK_memmap = np.memmap(
          filename=self.SOURCE_MASK_PATH, dtype=np.int64, mode="r", shape=(self.input_length,512,512)
        )

 
        self.TARGET_ID_memmap = np.memmap(
          filename=self.TARGET_ID_PATH, dtype=np.int64, mode="r", shape=(self.input_length,100)
        )
 
        self.TARGET_MASK_memmap = np.memmap(
          filename=self.TARGET_MASK_PATH, dtype=np.int64, mode="r",shape=(self.input_length,100)
        )

  
    def __len__(self):
        return self.list_index
  
    def __getitem__(self, index):
        return {"source_ids": torch.from_numpy(np.array(self.SOURCE_ID_memmap[index])).squeeze(), "source_mask": torch.from_numpy(np.array(self.SOURCE_MASK_memmap[index])).squeeze(), 
                "target_ids": torch.from_numpy(np.array(self.TARGET_ID_memmap[index])).squeeze(), "target_mask": torch.from_numpy(np.array(self.TARGET_MASK_memmap[index])).squeeze()}

## Training

In [8]:
import pytorch_lightning as pl
from mytransformers.src.transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
from torch.utils.data import DataLoader

from tqdm.auto import tqdm

import pandas as pd
from core.evaluation.eval import QGEvalCap

from omegaconf import DictConfig

def saveOutputs(hparams: DictConfig, inputs: list, outputs: list, targets: list) -> None:
  data = pd.DataFrame(list(zip(inputs, outputs, targets)), columns =['inputs', 'outputs', 'targets'])
  data.to_csv(os.path.join("out",hparams.experiment.model_dir.split("/")[-1]+".csv"),index=False, header=True)

def run_eval(target_src, decoded_text) -> dict:
  assert len(target_src) == len(decoded_text)
  eval_targets = {}
  eval_predictions = {}
  for idx in range(len(target_src)):
      eval_targets[idx] = [target_src[idx]]
      eval_predictions[idx] = [decoded_text[idx]]

  QGEval = QGEvalCap(eval_targets, eval_predictions)
  scores = QGEval.evaluate()
  return scores

class T5FineTuner(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(config.experiment.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(config.experiment.tokenizer_name_or_path, is_fast=True)

        special_tokens_dict = {'sep_token': '<sep>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None, mode=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
            three_dim_attention_mask=True
        )

    def _step(self, batch, mode=None):
        """ロス計算"""
        labels = batch["target_ids"].detach().clone()

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels,
            mode=mode
        )

        return outputs

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        outputs = self._step(batch)
        loss = outputs[0]
        logit = outputs[1]
        # logging metrics we calculated by hand
        self.log('train/loss', loss, on_epoch=True)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        outputs = self._step(batch)
        loss = outputs[0]
        logit = outputs[1]
        self.log("valid/loss_epoch", loss)  # default on val/test is on_epoch only
        # return {"val_loss": loss}

    def test_epoch_end(self, training_step_outputs):
        saveOutputs(self.config, self.inputs, self.outputs, self.targets)
        scores = run_eval(self.targets, self.outputs)
        wandb.log(scores)

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.config.training.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.config.training.learning_rate, 
                          eps=self.config.training.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.config.training.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args, mode=None):
        """データセットを作成する"""
        if args.experiment.data=="mhqg-wq":
          return JsonDatasetWQ(
            tokenizer=tokenizer, 
            data_dir=args.experiment.data_dir, 
            type_path=type_path, 
            input_max_len=args.model.max_input_length,
            target_max_len=args.model.max_target_length,
            mode=mode)
        else:
          return JsonDatasetPQ(
            tokenizer=tokenizer, 
            data_dir=args.experiment.data_dir, 
            type_path=type_path, 
            input_max_len=args.model.max_input_length,
            target_max_len=args.model.max_target_length,
            mode=mode)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.json", args=self.config, mode="Train")
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="dev.json", args=self.config, mode="Val")
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.config.training.train_batch_size * max(1, self.config.training.n_gpu)))
                // self.config.training.gradient_accumulation_steps
                * float(self.config.training.num_train_epochs)
            )
        elif stage == 'test':
            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="test.json", args=self.config, mode="Test")
            self.test_dataset = test_dataset

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.config.training.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.config.training.eval_batch_size, 
                          num_workers=4)
        
    def test_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.config.training.test_batch_size, 
                          num_workers=4)

## Evaluation

In [9]:
from mytransformers.src.transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
from torch.utils.data import DataLoader

from tqdm.auto import tqdm

import pandas as pd
from core.evaluation.eval import QGEvalCap

from omegaconf import DictConfig


# OmegaConf.register_new_resolver("now", lambda pattern: strftime(pattern, localtime()))

def saveOutputs(hparams: DictConfig, inputs: list, outputs: list, targets: list) -> None:
  data = pd.DataFrame(list(zip(inputs, outputs, targets)), columns =['inputs', 'outputs', 'targets'])
  data.to_csv(os.path.join("out",hparams.experiment.model_dir.split("/")[-1]+".csv"),index=False, header=True)

def run_eval(target_src, decoded_text) -> dict:
  assert len(target_src) == len(decoded_text)
  eval_targets = {}
  eval_predictions = {}
  for idx in range(len(target_src)):
      eval_targets[idx] = [target_src[idx]]
      eval_predictions[idx] = [decoded_text[idx]]

  QGEval = QGEvalCap(eval_targets, eval_predictions)
  scores = QGEval.evaluate()
  return scores

class T5Evaluation(pl.LightningModule):
    def __init__(self, config: DictConfig, train_params: DictConfig):
        super().__init__()
        self.config = config
        self.train_params = train_params
        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(self.config.experiment.model_dir)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(self.config.experiment.model_dir, is_fast=True)
        special_tokens_dict = {'sep_token': '<sep>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<answer>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<subject>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<relation>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        special_tokens_dict = {'sep_token': '<object>'}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        self.save_hyperparameters()

        self.inputs = []
        self.outputs = []
        self.targets = []


    def forward(self, input_ids, attention_mask=None, cross_attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model.generate(input_ids=input_ids, 
            attention_mask=attention_mask, 
            cross_attention_mask=cross_attention_mask,
            max_length=self.config.model.max_target_length,
            temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
            repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
            three_dim_attention_mask=True
            )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            cross_attention_mask=batch["cross_attention_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        return outputs

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        output = self._step(batch)
        labels = batch["target_ids"]
        labels[labels[:, :] == -100] = 0
        output_text = [self.tokenizer.decode(ids, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=False) 
                    for ids in output]
        target_text = [self.tokenizer.decode(ids, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=False) 
                    for ids in labels]
        input_text = [self.tokenizer.decode(ids, skip_special_tokens=False, 
                                clean_up_tokenization_spaces=False) 
                    for ids in batch["source_ids"]]

        self.inputs.extend(input_text)
        self.outputs.extend(output_text)
        self.targets.extend(target_text)

        return {"batch_idx":batch_idx}
        
    
    def test_epoch_end(self, training_step_outputs):
        saveOutputs(self.config, self.inputs, self.outputs, self.targets)
        scores = run_eval(self.targets, self.outputs)
        wandb.log(scores)

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.config.training.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.config.training.learning_rate, 
                          eps=self.config.training.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.config.training.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, mode=None):
        """データセットを作成する"""
        if self.config.get('experiment').data=="mhqg-wq":
          return JsonDatasetWQ(
            tokenizer=tokenizer, 
            data_dir=self.config.experiment.data_dir, 
            type_path=type_path, 
            input_max_len=self.config.model.max_input_length,
            target_max_len=self.config.model.max_target_length,
            mode=mode)
        else:
          return JsonDatasetPQ(
            tokenizer=tokenizer, 
            data_dir=self.config.experiment.data_dir, 
            type_path=type_path, 
            input_max_len=self.config.model.max_input_length,
            target_max_len=self.config.model.max_target_length,
            mode=mode)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'test':
            test_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="test.json", mode="Test")
            self.test_dataset = test_dataset
        
    def test_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.test_dataset, 
                          batch_size=self.config.training.test_batch_size, 
                          num_workers=4)

class Evaluation():
  def __init__(self, config, train_params):
    self.config = config
    self.train_params = train_params

  def run(self):
    # Tokenizer
    tokenizer = T5Tokenizer.from_pretrained(self.config.experiment.model_dir, is_fast=True)
    trained_model = T5ForConditionalGeneration.from_pretrained(self.config.experiment.model_dir)

    if self.config.experiment.data=="mhqg-wq":
        # import test data
        test_dataset = JsonDatasetWQ(tokenizer, self.config.experiment.data_dir, "test.json", 
                                input_max_len=self.config.model.max_input_length, 
                                target_max_len=self.config.model.max_target_length,
                                mode="Test")

    test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

    trained_model.eval()

    inputs = []
    outputs = []
    targets = []

    for index, batch in enumerate(tqdm(test_loader)):
        input_ids = batch['source_ids']
        input_mask = batch['source_mask']
        input_cross_attention_mask = batch["cross_attention_mask"]
        if self.config.training.n_gpu:
            input_ids = input_ids.cuda()
            input_mask = input_mask.cuda()

        output = trained_model.generate(input_ids=input_ids, 
            attention_mask=input_mask, 
            cross_attention_mask=input_cross_attention_mask,
            max_length=self.config.model.max_target_length,
            temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
            repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
            three_dim_attention_mask=True
            )

        output_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=False) 
                    for ids in output]
        target_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=False) 
                    for ids in batch["target_ids"]]
        input_text = [tokenizer.decode(ids, skip_special_tokens=False, 
                                clean_up_tokenization_spaces=False) 
                    for ids in input_ids]

        inputs.extend(input_text)
        outputs.extend(output_text)
        targets.extend(target_text)

        saveOutputs(self.config, inputs, outputs, targets)
        scores = run_eval(targets, outputs)
        self.train_params["logger"].log_metrics(scores)

## Main

In [10]:
import argparse
import glob
import os
import json
import time
import logging
import random
from itertools import chain
from string import punctuation
 
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from collections import Counter, defaultdict, OrderedDict
from nltk.tokenize import wordpunct_tokenize, word_tokenize
 
import gc

from omegaconf import DictConfig
import omegaconf

import hydra
import pytorch_lightning as pl



import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_train_param(config: DictConfig) -> dict:
    wandb_logger = WandbLogger(
        name=("exp_" + str(config.experiment.wandb.exp_num)),
        project=config.experiment.wandb.project,
        log_model=True,
    )
    checkpoint_path = os.path.join(
        config.experiment.model_dir, config.experiment.wandb.checkpoint_path
    )
    wandb_logger.log_hyperparams(config)
    early_stopping = EarlyStopping('valid/loss_epoch')
    train_params = dict(
        callbacks=[early_stopping],
        logger = wandb_logger,
        accumulate_grad_batches=config.training.gradient_accumulation_steps,
        gpus=config.training.n_gpu,
        max_epochs=config.training.num_train_epochs,
        precision= 16 if config.training.fp_16 else 32,
        amp_level=config.training.opt_level,
        gradient_clip_val=config.training.max_grad_norm,
    )
    return train_params

# @hydra.main(config_path="config.yaml")
def train(config: DictConfig, train_params: dict) -> None:
    
    # conduct transfer learning
    model = T5FineTuner(config)
    wandb.watch(model, log_freq=100)
    trainer = pl.Trainer(**train_params)
    trainer.fit(model)

    model.tokenizer.save_pretrained(config.experiment.model_dir)
    model.model.save_pretrained(config.experiment.model_dir)

    # trainer.test(model)

    # torch.save(
    #             {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth"
    #         )

# @hydra.main(config_path="adjacencyattentionwithoutselfloop", config_name="config")
def evaluation(config: DictConfig, train_params: dict) -> None:
    print("config",config)
    print("type_config",type(config))
    set_seed(42)
    USE_GPU = torch.cuda.is_available()
    config.training.n_gpu=1 if USE_GPU else 0
    eval_model = T5Evaluation(config, train_params)
    trainer = pl.Trainer(**train_params)
    trainer.test(eval_model)
  
def main(config: DictConfig) -> None:
    if os.path.exists(config.experiment.model_dir):
        raise "model_dir is exist"

    set_seed(42)
    USE_GPU = torch.cuda.is_available()
    config.training.n_gpu=1 if USE_GPU else 0
    train_params = get_train_param(config)

    Preprocess(config)
    train(config, train_params)
    evaluation(config, train_params)


In [11]:
!mkdir processed/mhqg-wq

In [ ]:
config = omegaconf.OmegaConf.load("config/config_test_pq.yaml")
main(config)
wandb.finish()

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [16]:
config = omegaconf.OmegaConf.load("config/config_test_wq.yaml")
main(config)
wandb.finish()

0 was finished 0
500 was finished 500
1000 was finished 1000
1500 was finished 1500


GPU available: False, used: False
TPU available: None, using: 0 TPU cores


FileNotFoundError: ignored

In [ ]:
config = omegaconf.OmegaConf.load("config/config_test_pq.yaml")
train_params = get_train_param(config)
evaluation(config, train_params)

In [ ]:
pwd

'/content/drive/My Drive/ColabNotebooks/Research/KG2QGwithT5/TripleAttention'